# Crear Ambiente de pruebas

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!nvidia-smi

Mon Mar  8 13:19:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
1.15.2


In [ ]:
!git clone https://github.com/tensorflow/models.git
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Cloning into 'models'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 53011 (delta 3), reused 13 (delta 1), pack-reused 52996
Receiving objects: 100% (53011/53011), 569.36 MiB | 30.05 MiB/s, done.
Resolving deltas: 100% (36312/36312), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources

In [ ]:
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [ ]:
import os
os.environ['PYTHONPATH'] += ':/models/research:/models/research/slim'

In [ ]:
!python setup.py build
!python setup.py install
!cp object_detection/packages/tf1/setup.py .
!python -m pip install .
!python object_detection/builders/model_builder_tf1_test.py

python3: can't open file 'setup.py': [Errno 2] No such file or directory
python3: can't open file 'setup.py': [Errno 2] No such file or directory
Processing /content/models/research
     |████████████████████████████████| 358kB 12.6MB/s 
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1624283 sha256=4ab4117fa207b6e6cd7dd41c4361f7846872179c0b51e8062eaac09399cb550e
  Stored in directory: /tmp/pip-ephem-wheel-cache-w76pdoo7/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] Mo

In [ ]:
%cd /content/

/content


# Montar Google Drive como carpeta

In [ ]:
from google.colab import drive,files
drive.mount('/content/drive')

Mounted at /content/drive


# Ubicación de modelos, videos y archivos

In [ ]:
# MODEL_PATH2 = modelo de caída de dientes
MODEL_PATH2 = '/content/drive/MyDrive/MineSense/Execution/Hudbay/Caida/frozen_inference_graph.pb'
# MODEL_PATH = modelo de desgate
MODEL_PATH = '/content/drive/MyDrive/MineSense/Execution/Wear/wear_base_31ene21.pb'
# VIDEO_PATH = video a probar
VIDEO_PATH = '/content/drive/MyDrive/MineSense/Execution/Hudbay/Videos/Termica/video000002.ts'
# SAVE_PATH = nombre de archivo de video procesado (.avi)
SAVE_PATH = '/content/drive/MyDrive/MineSense/Execution/Results/hudbay_caida.avi'
# FILE_PATH = ubicación de creación de archivos csv
FILE_PATH = '/content/drive/MyDrive/MineSense/Execution/Results/'

# Ejecución de modelos y guardado de video

In [ ]:
from PIL import Image
import tensorflow as tf
import numpy as np
import time
import socket
from struct import pack
from struct import unpack
import math
import cv2
import csv
from datetime import datetime
import os.path

In [ ]:
def filter_boxes_and_classes_by_score(boxes,scores,classes,min_score):
    filtered_boxes = []
    filtered_scores = []
    filtered_classes = []
    num_teeth = 6
    for i in range(0,len(boxes)):
        if scores[i] >= min_score:
            filtered_boxes.append(boxes[i])
            filtered_scores.append(scores[i])
            filtered_classes.append(classes[i])
        if len(filtered_boxes) >= num_teeth:
            break
    filtered_boxes = np.array(filtered_boxes)
    filtered_scores = np.array(filtered_scores)
    return filtered_boxes, filtered_scores, filtered_classes

In [ ]:
input_names = ['image_tensor']

detect_graph = tf.Graph()
with detect_graph.as_default():
    graph_def = tf.GraphDef()
    with tf.gfile.GFile(MODEL_PATH, 'rb') as fid:
        serialized_graph = fid.read()
        graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(graph_def, name='')
# define input/output tensors
tf_input = detect_graph.get_tensor_by_name('image_tensor:0')
tf_boxes = detect_graph.get_tensor_by_name('detection_boxes:0')
tf_scores = detect_graph.get_tensor_by_name('detection_scores:0')
tf_classes = detect_graph.get_tensor_by_name('detection_classes:0')
tf_num_detections = detect_graph.get_tensor_by_name('num_detections:0')

# create the session for inferencing
tf_sess = tf.Session(graph=detect_graph)

detect_graph2 = tf.Graph()
with detect_graph2.as_default():
    graph_def = tf.GraphDef()
    with tf.gfile.GFile(MODEL_PATH2, 'rb') as fid:
        serialized_graph = fid.read()
        graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(graph_def, name='')
# define input/output tensors
tf_input2 = detect_graph2.get_tensor_by_name('image_tensor:0')
tf_boxes2 = detect_graph2.get_tensor_by_name('detection_boxes:0')
tf_scores2 = detect_graph2.get_tensor_by_name('detection_scores:0')
tf_classes2 = detect_graph2.get_tensor_by_name('detection_classes:0')
tf_num_detections2 = detect_graph2.get_tensor_by_name('num_detections:0')

# create the session for inferencing
tf_sess2 = tf.Session(graph=detect_graph2)

In [ ]:
import math
def distance_bboxCenter_px(center1,center2):
    
    x1,y1 = center1
    x2,y2 = center2

    distance = math.sqrt(((x1-x2)**2) + ((y1-y2)**2))

    return distance


In [ ]:
def fits_zones_fall(image,boxes,classes,past_centers):
    movimiento = [1,1,1,1,1,1]
    distancias = [-1,-1,-1,-1,-1,-1]
    static_counter = 0

    font = cv2.FONT_HERSHEY_SIMPLEX 
    # fontScale 
    fontScale = 0.4
    # Red color in BGR 
    color = (0, 0, 255) 
    # Line thickness of 1 px 
    thickness = 1

    limit_size = 6
    # min_limit = [0,50,115,170,226,290]
    # max_limit = [45,105,165,220,285,320]

    min_limit = [0,43,105,165,215,278]
    max_limit = [40,100,160,210,275,320]

    # min_limit = [15,64,123,185,226,276]
    # max_limit = [60,114,165,222,270,320]

    detected_centers = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
    
    teeth_detected = [0,0,0,0,0,0]
    broken_detected = [0,0,0,0,0,0]
    number_teeth_detected = 0
    fall_teeth_detected = 0

    #Draw limits
    image = cv2.line(image, (0,90),(image.shape[1],90),(0,255,0),2)
    image = cv2.line(image, (0,256),(image.shape[1],256),(0,255,0),2)

    for i in range(0,limit_size):
        image = cv2.line(image, (min_limit[i],90),(min_limit[i],256),(i*30,i*30,255),2)
        image = cv2.line(image, (max_limit[i],90),(max_limit[i],256),(i*30,i*30,255),2)
    
    #Put numbers

    for k in range(len(boxes)):
        
        box = boxes[k] * np.array([image.shape[0], image.shape[1], image.shape[0], image.shape[1]])
        
        if classes[k] == 1:
            teeth_detected[i] = 1
            #number_teeth_detected = number_teeth_detected + 1
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 0.5
            color = (255, 0, 0)
            thickness = 1
            image = cv2.rectangle(image,(int(box[1]),int(box[0])),(int(box[3]),int(box[2])),(255,0,0), 2)
            image = cv2.putText(image, "normal", (int(box[1]),int(box[0]-5)), font,fontScale, color, thickness, cv2.LINE_AA)
        if classes[k] == 2:
            broken_detected[i] = 1
            #fall_teeth_detected = fall_teeth_detected + 1
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 0.5
            color = (0, 0, 255)
            thickness = 1
            image = cv2.rectangle(image,(int(box[1]),int(box[0])),(int(box[3]),int(box[2])),(0,255,255), 2)
            image = cv2.putText(image, "broken", (int(box[1]),int(box[0]-5)), font,fontScale, color, thickness, cv2.LINE_AA)
        
        y_min,x_min,y_max,x_max = box

        if y_min >= 90 and y_max <= 256:

            for i in range(0,limit_size):
                if x_min >= min_limit[i] and x_max <= max_limit[i]:
                    
                    if classes[k] == 1:
                        teeth_detected[i] = 1
                        number_teeth_detected = number_teeth_detected + 1
                        font = cv2.FONT_HERSHEY_SIMPLEX
                        fontScale = 0.5
                        color = (255, 0, 0)
                        thickness = 1
                        image = cv2.rectangle(image,(int(box[1]),int(box[0])),(int(box[3]),int(box[2])),(255,0,255), 2)
                        image = cv2.putText(image, "normal", (int(box[1]),int(box[0]-5)), font,fontScale, color, thickness, cv2.LINE_AA)


                        aux_center = [int((x_max-x_min)/2),int((y_max-y_min)/2)]
                        
                        #print("Aux Center: " + str(aux_center))
                        #print("Past Center: " + str(past_centers[i]))
                        if past_centers[i] != [0,0]:
                            distance = distance_bboxCenter_px(past_centers[i],aux_center)
                            distancias[i] = distance
                            #print("Distance: " + str(distance))
                            if distance <= 2:
                                movimiento[i] = 0
                        
                        past_centers[i] = aux_center
                        
                          
                    if classes[k] == 2:
                        broken_detected[i] = 1
                        fall_teeth_detected = fall_teeth_detected + 1
                        font = cv2.FONT_HERSHEY_SIMPLEX
                        fontScale = 0.5
                        color = (0, 0, 255)
                        thickness = 1
                        image = cv2.rectangle(image,(int(box[1]),int(box[0])),(int(box[3]),int(box[2])),(0,0,255), 2)
                        image = cv2.putText(image, "broken", (int(box[1]),int(box[0]-5)), font,fontScale, color, thickness, cv2.LINE_AA)
                    
    return image,teeth_detected,number_teeth_detected,broken_detected,fall_teeth_detected,movimiento,distancias
    
def filter_boxes_by_score(boxes,scores,min_score):
    filtered_boxes = []
    filtered_scores = []
    num_teeth = 6
    for i in range(0,len(boxes)):
        if scores[i] >= min_score:
            filtered_boxes.append(boxes[i])
            filtered_scores.append(scores[i])
        if len(filtered_boxes) >= num_teeth:
            break
    filtered_boxes = np.array(filtered_boxes)
    filtered_scores = np.array(filtered_scores)
    return filtered_boxes, filtered_scores
def order_boxes_and_scores(boxes,scores):
    boxes = boxes[0:6]
    scores = scores[0:6]

    scores2 = np.expand_dims(scores, axis=1)
    boxes_and_scores = np.append(boxes,scores2, axis=1)
    boxes_and_scores_sorted = np.asarray(sorted(boxes_and_scores, key=lambda x: x[1]))

    boxes = boxes_and_scores_sorted[:,0:4]
    scores = boxes_and_scores_sorted[:,4]
    return boxes, scores
def fits_zones(image,boxes,past_teeth):

    font = cv2.FONT_HERSHEY_SIMPLEX 
    # fontScale 
    fontScale = 0.5
    # Red color in BGR 
    color = (0, 0, 255) 
    # Line thickness of 1 px 
    thickness = 1

    limit_size = 6
    # min_limit = [0,50,115,170,226,290]
    # max_limit = [45,105,165,220,285,320]

    min_limit = [0,43,105,165,215,278]
    max_limit = [40,100,160,210,275,320]

    # min_limit = [15,64,123,185,226,276]
    # max_limit = [60,114,165,222,270,320]

    teeth_detected = [0,0,0,0,0,0]
    teeth_sizes = past_teeth.copy()
    number_teeth_detected = 0
    
    for k in range(len(boxes)):
        
        box = boxes[k] * np.array([image.shape[0], image.shape[1], image.shape[0], image.shape[1]])
        
        image = cv2.rectangle(image,(int(box[1]),int(box[0])),(int(box[3]),int(box[2])),(255,0,0), 2)
        
        y_min,x_min,y_max,x_max = box

        if y_min >= 90 and y_max <= 256:

            for i in range(0,limit_size):

                if x_min >= min_limit[i] and x_max <= max_limit[i]:

                    teeth_sizes[i] = int(box[2]) - int(box[0])
                    teeth_size = int(box[2]) - int(box[0])
                    
                    teeth_detected[i] = 1
                    number_teeth_detected = number_teeth_detected + 1 

                    
                    image = cv2.rectangle(image,(int(box[1]),int(box[0])),(int(box[3]),int(box[2])),(0,255,0), 2)
    
    return image,teeth_detected,number_teeth_detected,teeth_sizes

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
width_original = int(cap.get(3))
height_original = int(cap.get(4))
out = cv2.VideoWriter(SAVE_PATH,fourcc, 30.0, (width_original,height_original))
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [ ]:
class Communicator:
    def __init__(self):
        #self.videoPort = videoPort
        #self.dataPort = dataPort
        self.all_teeth_size = [0,0,0,0,0,0]
        self.scale = 254
        self.teeth_sizes = [self.scale,self.scale,self.scale,self.scale,self.scale,self.scale]
        self.teeth_sizes_filtered = [self.scale,self.scale,self.scale,self.scale,self.scale,self.scale]
        self.movimiento = [1,1,1,1,1,1]
        self.static_counter = [0,0,0,0,0,0]
        self.bucket_movements = 0
        self.teeth0 = []
        self.teeth1 = []
        self.teeth2 = []
        self.teeth3 = []
        self.teeth4 = []
        self.teeth5 = []
        self.state_teeth = [1,1,1,1,1,1]
        self.total_frame_teeth = [0,0,0,0,0,0]
        self.static_counter = [0,0,0,0,0,0]
        self.past_centers = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]

In [ ]:
frame_count = 0
serverComm = Communicator()
np.set_printoptions(suppress=True)

font = cv2.FONT_HERSHEY_SIMPLEX 
fontScale = 0.4
color = (0, 0, 255)
thickness = 1

if os.path.isfile(FILE_PATH + 'caida_boxes_raw.csv'):
    print ("File exist")
else:
    with open(FILE_PATH + 'caida_boxes_raw.csv', 'w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(["Date", "Time","Boxes","Scores","Classes","Num. Detections"])

if os.path.isfile(FILE_PATH + 'desgaste_boxes_raw.csv'):
    print ("File exist")
else:
    with open(FILE_PATH + 'desgaste_boxes_raw.csv', 'w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(["Date", "Time","Boxes","Scores","Classes","Num. Detections"])

if os.path.isfile(FILE_PATH + 'teeth_lenght_px_filtered.csv'):
    print ("File exist")
else:
    with open(FILE_PATH + 'teeth_lenght_px_filtered.csv', 'w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(["Date", "Time","Teeth0","Teeth1","Teeth2","Teeth3","Teeth4","Teeth5"])

if os.path.isfile(FILE_PATH + 'teeth_lenght_px_raw.csv'):
    print ("File exist")
else:
    with open('teeth_lenght_px_raw.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Time","Teeth0","Teeth1","Teeth2","Teeth3","Teeth4","Teeth5"])

if os.path.isfile(FILE_PATH + 'teeth_lenght_px_raw_extraData.csv'):
    print ("File exist")
else:
    with open(FILE_PATH + 'teeth_lenght_px_raw_extraData.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Time","Teeth0","Teeth1","Teeth2","Teeth3","Teeth4","Teeth5","number_teeth_detected","array_teeth"])

if os.path.isfile(FILE_PATH + 'teeth_normal.csv'):
    print ("File exist")
else:
    with open(FILE_PATH + 'teeth_normal.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Time","Teeth0","Teeth1","Teeth2","Teeth3","Teeth4","Teeth5","number_teeth_detected"])

if os.path.isfile(FILE_PATH + 'teeth_broken.csv'):
    print ("File exist")
else:
    with open(FILE_PATH + 'teeth_broken.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Time","Teeth0","Teeth1","Teeth2","Teeth3","Teeth4","Teeth5","number_broken_detected"])

if os.path.isfile(FILE_PATH + 'bucket_movement.csv'):
    print ("File exist")
else:
    with open(FILE_PATH + 'bucket_movement.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Time","Dist_Teeth0","Dist_Teeth1","Dist_Teeth2","Dist_Teeth3","Dist_Teeth4","Dist_Teeth5","number_teeth_detected","teeth_detected","static_counter","bucket_moving"])
        
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:

      #Resize image
      image_resized = cv2.resize(frame, (300, 300))
      
      #Run detection
      scores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections], feed_dict={
              tf_input: image_resized[None, ...]
          })
      
      scores2, boxes2, classes2, num_detections2 = tf_sess2.run([tf_scores2, tf_boxes2, tf_classes2, tf_num_detections2], feed_dict={
          tf_input2: image_resized[None, ...]
      })

      #new_frame = frame.copy()

      boxes2 = boxes2[0] 
      scores2 = scores2[0]
      classes2 = classes2[0]
      num_detections2 = num_detections2[0]

      now = datetime.now()
      date = now.strftime("%d/%m/%Y")
      time = now.strftime("%H:%M:%S")
      
      with open(FILE_PATH + 'caida_boxes_raw.csv', 'a', newline='') as file:
          writer = csv.writer(file)
          writer.writerow([date,time,boxes2,scores2,classes2,num_detections2])

      #boxes2, scores2 = filter_boxes_by_score(boxes2,scores2,min_score=0.5)
      boxes2, scores2, classes2 = filter_boxes_and_classes_by_score(boxes2,scores2,classes2,min_score=0.5)


      #######################################  FALL DETECTION #############################################

      new_frame = frame.copy()
      
      new_frame,teeth_detected,number_teeth_detected, broken_detected,number_broken_detected,movimiento,distancias = fits_zones_fall(new_frame,boxes2,classes2,serverComm.past_centers)
      serverComm.movimiento = movimiento



      for m in range(len(serverComm.movimiento)):
          if serverComm.movimiento[m] == 0:
              if serverComm.static_counter[m] < 1000:
                  serverComm.static_counter[m] = serverComm.static_counter[m] + 1
          else:
              serverComm.static_counter[m] = 0
          
      static_value = 0
      for m in range(len(serverComm.static_counter)):
          if serverComm.static_counter[m] >= 120:
              static_value = static_value + 1
      
      bucket_moving = 1
      if static_value >= 5:
          bucket_moving = 0
          new_frame = cv2.putText(new_frame, "STATIC", (200,28), font,fontScale, color, thickness, cv2.LINE_AA)
      else:
          bucket_moving = 1
          new_frame = cv2.putText(new_frame, "MOVING", (200,28), font,fontScale, color, thickness, cv2.LINE_AA)
      
      now = datetime.now()
      date = now.strftime("%d/%m/%Y")
      time = now.strftime("%H:%M:%S")
      
      with open(FILE_PATH + 'bucket_movement.csv', 'a', newline='') as file:
          writer = csv.writer(file)
          writer.writerow([date,time,distancias[0],distancias[1],distancias[2],distancias[3],distancias[4],distancias[5],number_teeth_detected,teeth_detected,serverComm.static_counter,bucket_moving])
              
      

      #print("=================================")
      #print("Movimiento: " + str(movimiento))
      #print("=================================")

    
      


      font = cv2.FONT_HERSHEY_SIMPLEX 
      fontScale = 0.4
      color = (0, 0, 255)
      thickness = 1
      new_frame = cv2.putText(new_frame, "inst: " + str(teeth_detected), (10,58), font,fontScale, color, thickness, cv2.LINE_AA)
      new_frame = cv2.putText(new_frame, "# teeth: " + str(number_teeth_detected), (200,58), font,fontScale, color, thickness, cv2.LINE_AA)
      
      if bucket_moving == 1:
          if number_teeth_detected >= 5:
              for i in range(len(teeth_detected)):
                  if (teeth_detected[i]==0 and serverComm.state_teeth[i] == 1) or (teeth_detected[i]==1 and serverComm.state_teeth[i] == 0):
                      serverComm.total_frame_teeth[i] = serverComm.total_frame_teeth[i] + 1
                  elif (teeth_detected[i]==1 and serverComm.state_teeth[i] == 1) or (teeth_detected[i]==0 and serverComm.state_teeth[i] == 0):
                      serverComm.total_frame_teeth[i] = 0
          if number_broken_detected > 0:
              for i in range(len(broken_detected)):
                  if (broken_detected[i]==1 and serverComm.state_teeth[i] == 1):
                      serverComm.total_frame_teeth[i] = serverComm.total_frame_teeth[i] + 1
                  #elif (teeth_detected[i]==1 and serverComm.state_teeth[i] == 1) or (teeth_detected[i]==0 and serverComm.state_teeth[i] == 0):
                      #serverComm.total_frame_teeth[i] = 0   

      for i in range(len(teeth_detected)):
          if serverComm.total_frame_teeth[i]>100 and serverComm.state_teeth[i]==1:
              serverComm.state_teeth[i] = 0
              serverComm.total_frame_teeth[i] = 0
              #alertas = alertas + 1
          elif serverComm.total_frame_teeth[i]>100 and serverComm.state_teeth[i]==0:
              serverComm.state_teeth[i] = 1
              serverComm.total_frame_teeth[i] = 0
              #reposicion = reposicion + 1

      ####################################### END FALL DETECTION #############################################


      boxes = boxes[0] 
      scores = scores[0]
      classes = classes[0]
      num_detections = num_detections[0]

      now = datetime.now()
      date = now.strftime("%d/%m/%Y")
      time = now.strftime("%H:%M:%S")
      
      with open(FILE_PATH + 'desgaste_boxes_raw.csv', 'a', newline='') as file:
          writer = csv.writer(file)
          writer.writerow([date,time,boxes,scores,classes,num_detections])

      boxes_filtered, scores_filtered = filter_boxes_by_score(boxes,scores,min_score=0.4)
      
      #frame = draw_zones(frame)
      if (len(boxes_filtered) > 0) and (len(scores_filtered) > 0):

          boxes_ordered,scores_ordered = order_boxes_and_scores(boxes_filtered,scores_filtered)
          #frame = detector.draw_bb(frame,boxes_ordered,scores_ordered)

      
          new_frame, teeth_detected, number_teeth_detected, teeth_sizes = fits_zones(new_frame,boxes_ordered,serverComm.teeth_sizes)
          
          serverComm.teeth_sizes = teeth_sizes

          # if number_teeth_detected >=5 and serverComm.movimiento == 0:
          #     serverComm.movimiento = 1
          # elif number_teeth_detected == 0 and serverComm.movimiento == 1:
          #     serverComm.movimiento = 0
          #     serverComm.bucket_movements = serverComm.bucket_movements + 1
          
          # if serverComm.movimiento == 1:
          out_of_limits = 0
          for i in range(len(teeth_sizes)):
              if teeth_sizes[i] < 15 or teeth_sizes[i] > 35:
                  out_of_limits = 1

          if number_teeth_detected >= 5 and out_of_limits == 0:
              serverComm.teeth0.append(int(teeth_sizes[0]))
              serverComm.teeth1.append(int(teeth_sizes[1]))
              serverComm.teeth2.append(int(teeth_sizes[2]))
              serverComm.teeth3.append(int(teeth_sizes[3]))
              serverComm.teeth4.append(int(teeth_sizes[4]))
              serverComm.teeth5.append(int(teeth_sizes[5]))

              num_measurements = 9000
              if len(serverComm.teeth0) > num_measurements:
                  serverComm.teeth0.pop(0)
              if len(serverComm.teeth1) > num_measurements:
                  serverComm.teeth1.pop(0)
              if len(serverComm.teeth2) > num_measurements:
                  serverComm.teeth2.pop(0)
              if len(serverComm.teeth3) > num_measurements:
                  serverComm.teeth3.pop(0)
              if len(serverComm.teeth4) > num_measurements:
                  serverComm.teeth4.pop(0)
              if len(serverComm.teeth5) > num_measurements:
                  serverComm.teeth5.pop(0)
            
            
          if len(serverComm.teeth0) > 0:
              teeth0_arr = np.array(serverComm.teeth0, dtype=int)
              serverComm.teeth_sizes_filtered[0] = np.bincount(teeth0_arr).argmax()
          elif len(serverComm.teeth0) == 0:
              serverComm.teeth_sizes_filtered[0] = 254

          if len(serverComm.teeth1) > 0:
              teeth1_arr = np.array(serverComm.teeth1, dtype=int)
              serverComm.teeth_sizes_filtered[1] = np.bincount(teeth1_arr).argmax()
          elif len(serverComm.teeth1) == 0:
              serverComm.teeth_sizes_filtered[1] = 254

          if len(serverComm.teeth2) > 0:
              teeth2_arr = np.array(serverComm.teeth2, dtype=int)
              serverComm.teeth_sizes_filtered[2] = np.bincount(teeth2_arr).argmax()
          elif len(serverComm.teeth2) == 0:
              serverComm.teeth_sizes_filtered[2] = 254

          if len(serverComm.teeth3) > 0:
              teeth3_arr = np.array(serverComm.teeth3, dtype=int)
              serverComm.teeth_sizes_filtered[3] = np.bincount(teeth3_arr).argmax()
          elif len(serverComm.teeth3) == 0:
              serverComm.teeth_sizes_filtered[3] = 254

          if len(serverComm.teeth4) > 0:
              teeth4_arr = np.array(serverComm.teeth4, dtype=int)
              serverComm.teeth_sizes_filtered[4] = np.bincount(teeth4_arr).argmax()
          elif len(serverComm.teeth4) == 0:
              serverComm.teeth_sizes_filtered[4] = 254

          if len(serverComm.teeth5) > 0:
              teeth5_arr = np.array(serverComm.teeth5, dtype=int)
              serverComm.teeth_sizes_filtered[5] = np.bincount(teeth5_arr).argmax()
          elif len(serverComm.teeth5) == 0:
              serverComm.teeth_sizes_filtered[5] = 254
          
          font = cv2.FONT_HERSHEY_SIMPLEX 
          fontScale = 0.4
          color = (0, 0, 255)
          thickness = 1

          for i in range(len(serverComm.state_teeth)):
              if serverComm.state_teeth[i] == 0:
                  #frame = cv2.putText(frame, "fall", (10,34), font,fontScale, color, thickness, cv2.LINE_AA)
                  #new_frame = cv2.putText(new_frame, "original: " +  str(serverComm.teeth_sizes_filtered[i]) ,(10,14), font,fontScale, color, thickness, cv2.LINE_AA)
                  serverComm.teeth_sizes_filtered[i] = 255
                  #new_frame = cv2.putText(new_frame, "altered: " +  str(serverComm.teeth_sizes_filtered[i]) ,(10,24), font,fontScale, color, thickness, cv2.LINE_AA)

          font = cv2.FONT_HERSHEY_SIMPLEX 
          fontScale = 0.4
          color = (0, 0, 255)
          thickness = 1
          
          now = datetime.now()
          date = now.strftime("%d/%m/%Y")
          time = now.strftime("%H:%M:%S")

          

          with open(FILE_PATH + 'teeth_lenght_px_filtered.csv', 'a', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([date,time,serverComm.teeth_sizes_filtered[0],serverComm.teeth_sizes_filtered[1],serverComm.teeth_sizes_filtered[2],serverComm.teeth_sizes_filtered[3],serverComm.teeth_sizes_filtered[4],serverComm.teeth_sizes_filtered[5]])
              
          with open(FILE_PATH +'teeth_lenght_px_raw.csv', 'a', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([date,time,teeth_sizes[0],teeth_sizes[1],teeth_sizes[2],teeth_sizes[3],teeth_sizes[4],teeth_sizes[5]])
          
          with open(FILE_PATH +'teeth_lenght_px_raw_extraData.csv', 'a', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([date,time,teeth_sizes[0],teeth_sizes[1],teeth_sizes[2],teeth_sizes[3],teeth_sizes[4],teeth_sizes[5],number_teeth_detected,teeth_detected])

          with open(FILE_PATH +'teeth_normal.csv', 'a', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([date,time,teeth_detected[0],teeth_detected[1],teeth_detected[2],teeth_detected[3],teeth_detected[4],teeth_detected[5],number_teeth_detected])

          with open(FILE_PATH +'teeth_broken.csv', 'a', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([date,time,broken_detected[0],broken_detected[1],broken_detected[2],broken_detected[3],broken_detected[4],broken_detected[5],number_broken_detected])

      new_frame = cv2.putText(new_frame, "frame: " + str(serverComm.total_frame_teeth), (10,28), font,fontScale, color, thickness, cv2.LINE_AA)
      new_frame = cv2.putText(new_frame, "state: " + str(serverComm.state_teeth), (10,38), font,fontScale, color, thickness, cv2.LINE_AA)
      new_frame = cv2.putText(new_frame, "lenght: " + str(serverComm.teeth_sizes_filtered), (10,48), font,fontScale, color, thickness, cv2.LINE_AA)
      new_frame = cv2.putText(new_frame, "static_counter: " + str(serverComm.static_counter), (10,68), font,fontScale, color, thickness, cv2.LINE_AA)
        
      out.write(new_frame)
    else:
        break

print("~~~~~~~~ VIDEO PROCESSED ! ~~~~~~~~~~~")


cap.release()
out.release()

File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
~~~~~~~~ VIDEO PROCESSED ! ~~~~~~~~~~~


# Descargar video generado

In [ ]:
files.download(SAVE_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>